In [1]:
import pandas as pd
import nltk
import string
import re
import ast
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\azmik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

In [3]:
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [4]:
from keras.models import load_model

In [5]:
import json

# Preprocessing Function

In [6]:
def remove_punctuation(text):
  text = re.sub('-',' ',text)
  text = text.translate(str.maketrans('', '', string.punctuation))
  return text

def case_folding(text):
  text = text.lower()
  return text

def tokenizingText(text):
  text = word_tokenize(text)
  return text

def stemmingText(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  text = [stemmer.stem(word) for word in text]
  return text

def toSentence(list_words): # Convert list of words into sentence
  sentence = ' '.join(word for word in list_words)
  return sentence

In [7]:
def data_preprocessing(text):
  cleaned_text = remove_punctuation(text)
  cleaned_text = case_folding(cleaned_text)
  tokenized_text = tokenizingText(cleaned_text)
  stemmed_text = stemmingText(tokenized_text)
  preprocessed_text = toSentence(stemmed_text)

  return preprocessed_text

# Tokenizing Function

In [8]:
tokenizer = Tokenizer(split=' ', oov_token='<OOV>')
with open('word_index.json') as json_word_index:
  word_index = json.load(json_word_index)
tokenizer.word_index = word_index

In [9]:
def token_for_sequences(preprocessed_text):
  sequences = tokenizer.texts_to_sequences(preprocessed_text)
  padded_sequences = pad_sequences(sequences, padding='post', maxlen=100)

  return padded_sequences

# Model and Prediction

In [10]:
model = load_model('bidirectional_lstm.h5')

In [11]:
def preprocess_to_predict(text):
  preprocessed_text = data_preprocessing(text)
  padded_sequences = token_for_sequences([preprocessed_text])
  result = model.predict(padded_sequences)

  return result


In [12]:
preprocess_to_predict('Woofy agresif banget kalo lihat anjing lain. saya takut berantem sama anjing lain.')

1/1 [==============================] - 2s 2s/step


array([[0.44970438]], dtype=float32)